## 코드
### 아래에 정리 후 삭제

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5
 
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)
 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
 
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape
 
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # (주석 예) ? = 1, ? = 10, kernel size = 5, stribe = 1, ? = 0 인 ...
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.mp = nn.MaxPool2d(2) 
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) 
 
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
 
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        # (주석 예) 모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : ?
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
 
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

: 

: 

# 1. Import API

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

: 

-(import torch.nn) : 클래스로 정의되어 모델 정의에 쓰임, torch.nn으로 구현한 클래스의 
경우에는 attribute를 활용해 state를 저장하고 활용할 수 있다

-(import torch.nn.furncional) : 함수로 구성되어있고 따라서 이런 경우에는 인스턴스화 시킬 필요 없이 사용이 가능하다

-(torch.optim) : 최적화 알고리즘을 구현하는 패키지로써 사용하려면 현재 상태를 유지하고 계산 된 그래디언트를 기반으로 매개 변수를 업데이트하는 옵티 마이저 객체를 생성해야한다

-(torchvision.transforms) : 파이토치에서 공식적으로 제공하는 모듈이며 albumentations 는 OpenCV 와 같은 오픈 소스 컴퓨터 비젼 라이브러리를 최적화 하였기에 다른 라이브러리보다 더 빠른 처리 속도 및 기타 기능을 제공하는 모듈입니다.

-device = 'cuda' if torch.cuda.is_available() else 'cpu' :멀티 gpu중 0번 gpu에만 모든 cuda 연산을 할당

learning_rate = 0.001 : 경사하강법을 사용할 때의 가중치
batch_size = 100 : 총 데이터를 소 그룹으로 나누었을 때 한그룹의 데이터 수
num_classes = 10 
epochs = 5 : 신경망을 통과한 횟수


# 2. 데이터 불러오기

In [ ]:
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)
 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
 

: 

-Torchvision은 PyTorch와 함께 사용되는 Computer Vision 용 라이브러리 효율적인 이미지 및 비디오 변환을위해서 일반적으로 사용되는 일부 사전 학습 된 모델 및 일부 데이터 세트가 존재

-MNIST 데이셋 호출

-root : 데이터의 경로를 넣는다.

-train : 테스트용 데이터를 가져올지 학습용 데이터를 가져올지 표시한다. True면 테스트용 데이터이다.

-transform : 어떤 형태로 데이터를 불러올 것인가. 일반 이미지는 0-255사이의 값을 갖고, (H, W, C)의 형태를 갖는 반면 pytorch는 0-1사이의 값을 가지고 (C, H, W)의 형태를 갖는다. transform에 transforms.ToTensor()를 넣어서 일반 이미지(PIL image)를 pytorch tensor로 변환한다.

-download : True로 설정하면 MNIST 데이터가 없으면 다운을 받는다.

-Epoch는 training set 전체가 학습에 사용된 횟수를 말한다.

-batch_size는 전체 데이터를 자를 사이즈를 말한다.

-transforms.compose : 이미지 와 ndarray를 FlaotTensor로 변환 및 화소 밝음 정도 스케일링

# 3. train set 확인하기

In [ ]:
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

: 

-numerate() 함수는 인자로 넘어온 목록을 기준으로 인덱스와 원소를 차례대로 접근하게 해주는 반복자(iterator) 객체를 반환해주는 함수입니다.

-enumerate() 함수를 호출한 결과를 대상으로 next() 함수를 계속해서 호출해보면, 인덱스와 원소의 쌍이 튜플(tuple)의 형태로 차례로 얻어진다

-각각의 데이터의 shape 확인

# 4.(Convolutional Neural Network, ConvNet)

In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # (주석 예) ? = 1, ? = 10, kernel size = 5, stribe = 1, ? = 0 인 ...
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.mp = nn.MaxPool2d(2) 
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) 

: 

-첫번째 2D 합성곱 계층 : 1개의 입력 채널(이미지)을 받아들이고, 사각 커널 사이즈가 5인 10개의 합성곱 특징들을 출력합니다.

-두번쨰 2D 합성곱 계층 : 10개의 입력 계층을 받아드리고, 사각 커널 사이즈가 5인 20개의 합성곱 특징을 출력

-인접한 픽셀들은 입력 활률에 따라 모두 0값을 가지거나 혹은 모두 유효한 값이 되도록 만든다.

-첫번째 fully connected layer + 10개의 라벨을 출력하는 두번째 fully connected layer

-foward 에서 x는 데이터를 나타내고 데이터가 conv1,2을 지나간후 x를 ReLU 활성함수에 대입한다.

-flatten 후에

-데이터가 fc1,2 레이어를 을 지나간후에 softmax를 적용

# 5.Train model

In [ ]:
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
 
for epoch in range(epochs): #각각의 epcch(5)마다 반복하고
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
 

: 

-model = ConvNet().to(device) : ConvNet CLASS Model section

-criterion = nn.CrossEntropyLoss().to(device) : This criterion computes the cross entropy loss between input and target

-optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) :torch.optim is a package implementing various optimization algorithms.

**torch.optimate를 사용하려면 현재 상태를 유지하고 계산된 그라데이션에 따라 매개변수를 업데이트하는 옵티마이저 객체를 생성해야 합니다.

    for data, target in train_loader: #data,targe 가져오기
        data = data.to(device) 
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data) # 가정치,cost 계산 : (hypotesis 값 반환)
        cost = criterion(hypothesis, target) : # cost_function을 사용하여 cost 계산
        cost.backward() : gradient 측정
        optimizer.step() : 모델 매개변수 최적화하기
        avg_cost += cost / len(train_loader) :평균 계싼


# 6. Evaluation model // test accuracy

In [ ]:
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

: 

-eval() : 해당 모델의 모든 레이어가 evaluation mode에 들어가게한다. 학습할 떄만 Dropout, Batchnorm등의 기능을 비활성화시킨다.

-model.eval() : 추론할 때의 모드로 작동하도록 조정해준다

-no_grad() : gradient 게산 context를 비활성화 해주는 역할로써 inference나 validation을 할 때 주로 사용한다, -> 필요한 메모리가 줄어들고 연산속도 증가한다. 

-test_loader 데이터, 실제값 끌어오기

-모델을 통해서 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택

-예측값이 같으면 더한다

-accuracy 출력 : corret / total



: 